In [1]:
import sys

duhast_path = r"C:\Users\janchristel\Documents\GitHub\SampleCodeRevitBatchProcessor\src"
sys.path += [duhast_path]

In [2]:
from duHast.Revit.Views.Reporting.views_data_report import read_view_data_from_file
from duHast.Utilities.files_get import (
    get_files_single_directory,
    get_file_name_without_ext,
)
from duHast.Revit.Views.Reporting.views_data_hash_report import (
    _get_hash_headers,
    _get_hash_rows_categories,
    _get_hash_for_category_overrides,
)
from duHast.Revit.Views.Reporting.view_reports_json_props import PROP_FILE_NAME

JSON_DIRECTORY = r"C:\Users\janchristel\Documents\GitHub\SampleCodeRevitBatchProcessor\test\Data\Jupyter"

ModuleNotFoundError: No module named 'duHast'

In [ ]:
JSON_FILES = get_files_single_directory(
    folder_path=JSON_DIRECTORY, file_prefix="", file_suffix="", file_extension=".csv"
)
print(JSON_FILES)

Load json files


In [ ]:
def load_json_data(files):
    json_data = {}
    for file_path in files:
        file_name = get_file_name_without_ext(file_path=file_path)
        json_single_data = read_view_data_from_file(file_path=file_path)
        json_data[file_name] = json_single_data
    return json_data


JSON_DATA_LOADED = load_json_data(JSON_FILES)

print("found {} files".format(len(JSON_DATA_LOADED)))
print(type(JSON_DATA_LOADED))

define a storage class


In [ ]:
class json_three_d_storage():

    def __init__(self):
        self.column_headers = []
        self.row_headers = []
        self.hash_table = []
        self.padded_default_hash_table = []
        self.merged_column_headers=[]
        self.merged_row_headers = []
        self.column_indices = []
        self.row_indices=[]


get hash tables per file


In [ ]:
def get_hash_tables_by_file(view_settings):
    dic_tables_by_file = {}
    for key, vt_setting in view_settings.items():
        column_headers = _get_hash_headers(vt_setting)
        row_headers = _get_hash_rows_categories(vt_setting)
        hash_table = _get_hash_for_category_overrides(
            headers=column_headers, row_headers=row_headers, views_settings=vt_setting
        )
        storage = json_three_d_storage()
        storage.column_headers=column_headers
        storage.row_headers=row_headers
        storage.hash_table=hash_table
        dic_tables_by_file[key] = storage

    return dic_tables_by_file


HASH_DATA_BY_FILE = get_hash_tables_by_file(JSON_DATA_LOADED)
print(HASH_DATA_BY_FILE)

build overall headers and rows


In [ ]:
def combine_headers(view_settings, index):
    """_summary_

    Args:
        view_settings (_type_): _description_
        index (_type_): 0 is column header, 1 is row header

    Returns:
        _type_: _description_
    """
    overall_headers = []
    for key, vt_setting in view_settings.items():
        overall_headers = sorted(list(set(overall_headers) | set(vt_setting[index])))
    return overall_headers


MERGED_COLUMN_HEADERS = combine_headers(view_settings=HASH_DATA_BY_FILE, index=0)
print(
    "\n number of column headers: {}".format(len(MERGED_COLUMN_HEADERS)),
    MERGED_COLUMN_HEADERS,
)

MERGED_ROW_HEADERS = combine_headers(view_settings=HASH_DATA_BY_FILE, index=1)
print(
    "\n number of row headers: {}".format(len(MERGED_ROW_HEADERS)), MERGED_ROW_HEADERS
)

build hash tables


In [ ]:
def get_padded_default_array(
    merged_row_headers,
    merged_column_headers,
):
    # Create a new padded 2D array
    padded_array = [
        [-1 for entry in merged_column_headers] for entry in merged_row_headers
    ]
    return padded_array


for key, hash_by_file in HASH_DATA_BY_FILE.items():
    padded_array = get_padded_default_array(
        merged_column_headers=hash_by_file[0], merged_row_headers=hash_by_file[1]
    )
    HASH_DATA_BY_FILE[key].padded_default_hash_table= padded_array

In [ ]:
for key, hash_by_file in HASH_DATA_BY_FILE:
    # Find the indices for row and column headers in the merged headers
    ROW_INDICES_A = [MERGED_ROW_HEADERS.index(row) for row in hash_by_file[1]]
    COLUMN_INDICES_A = [MERGED_COLUMN_HEADERS.index(col) for col in hash_by_file[0]]
    
    HASH_DATA_BY_FILE[key].row_indices=ROW_INDICES_A
    HASH_DATA_BY_FILE[key].column_indices=COLUMN_INDICES_A

In [ ]:
def update_default_array_values(row_indices, col_indices, default_array, value_array):
    # Fill in the values from array_model_a
    for i, row_index in enumerate(row_indices):
        for j, col_index in enumerate(col_indices):
            default_array[row_index][col_index] = value_array[i][j]
    return default_array

for key, hash_by_file in HASH_DATA_BY_FILE.items():
    updated_array = update_default_array_values(row_indices=hash_by_file[], col_indices=hash_by_file[], default_array=hash_by_file[], value_array=hash_by_file[])